In [813]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import tree
import graphviz
from sklearn.model_selection import GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [814]:
df=pd.read_csv("weatherAUS.csv")
df.head(10)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
5,2008-12-06,Albury,14.6,29.7,0.2,NaN,NaN,WNW,56.0,W,...,55.0,23.0,1009.2,1005.4,NaN,NaN,20.6,28.9,No,No
6,2008-12-07,Albury,14.3,25.0,0.0,NaN,NaN,W,50.0,SW,...,49.0,19.0,1009.6,1008.2,1.0,NaN,18.1,24.6,No,No
7,2008-12-08,Albury,7.7,26.7,0.0,NaN,NaN,W,35.0,SSE,...,48.0,19.0,1013.4,1010.1,NaN,NaN,16.3,25.5,No,No
8,2008-12-09,Albury,9.7,31.9,0.0,NaN,NaN,NNW,80.0,SE,...,42.0,9.0,1008.9,1003.6,NaN,NaN,18.3,30.2,No,Yes
9,2008-12-10,Albury,13.1,30.1,1.4,NaN,NaN,W,28.0,S,...,58.0,27.0,1007.0,1005.7,NaN,NaN,20.1,28.2,Yes,No


In [815]:
'''. Preprocess the data, remove the attributes which were are not useful to predict 
rain. Also, remove rows with at least one missing value for each of them. '''

# df=df.drop(columns=['RainToday','RainTomorrow'])
# df.head(10)

df['Date'] = pd.DatetimeIndex(df['Date']).month
df.head(10)


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,12,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,12,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,12,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,12,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,12,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
5,12,Albury,14.6,29.7,0.2,NaN,NaN,WNW,56.0,W,...,55.0,23.0,1009.2,1005.4,NaN,NaN,20.6,28.9,No,No
6,12,Albury,14.3,25.0,0.0,NaN,NaN,W,50.0,SW,...,49.0,19.0,1009.6,1008.2,1.0,NaN,18.1,24.6,No,No
7,12,Albury,7.7,26.7,0.0,NaN,NaN,W,35.0,SSE,...,48.0,19.0,1013.4,1010.1,NaN,NaN,16.3,25.5,No,No
8,12,Albury,9.7,31.9,0.0,NaN,NaN,NNW,80.0,SE,...,42.0,9.0,1008.9,1003.6,NaN,NaN,18.3,30.2,No,Yes
9,12,Albury,13.1,30.1,1.4,NaN,NaN,W,28.0,S,...,58.0,27.0,1007.0,1005.7,NaN,NaN,20.1,28.2,Yes,No


In [816]:
df.describe()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,145460.000000,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,6.399615,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,3.427262,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,1.000000,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,3.000000,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,6.000000,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,9.000000,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,12.000000,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


In [817]:
df=df.dropna(axis=0)
df.head(10)
print(df['Location'].value_counts())

z=['Location','WindGustDir','WindDir9am','WindDir3pm',"RainToday","RainTomorrow"]
le = preprocessing.LabelEncoder()
df[z] = df[z].apply(le.fit_transform)

df['WindDir9am'].value_counts()

Darwin              3062
Perth               3025
Brisbane            2953
MelbourneAirport    2929
PerthAirport        2913
SydneyAirport       2870
Watsonia            2730
Mildura             2594
MountGambier        2465
NorfolkIsland       2464
Cairns              2444
Townsville          2419
WaggaWagga          2416
AliceSprings        2223
Nuriootpa           2008
Hobart              1939
Moree               1913
Melbourne           1898
Portland            1863
Woomera             1734
Sydney              1690
Sale                1678
CoffsHarbour        1380
Williamtown         1198
Canberra            1078
Cobar                534
Name: Location, dtype: int64


3     4967
0     4456
1     3932
10    3893
9     3880
13    3707
8     3421
2     3400
4     3390
12    3356
5     3267
15    3037
6     3016
11    2926
14    2918
7     2854
Name: WindDir9am, dtype: int64

In [818]:
x2=df.drop(columns="Rainfall",axis=1).values
y2=df['Rainfall'].values


x_train, x_test, y_train, y_test = train_test_split(x2, y2, test_size=0.3, random_state=1)


lm2 = LinearRegression()
lm2.fit(x_train,y_train)

y_predicted2=lm2.predict(x_test)

print("R2 score: %.2f" % r2_score(y_test,y_predicted2))
print("Mean absolute error  %.2f" %mean_absolute_error(y_test,y_predicted2))
print("Residual sum of squares %.2f"
  % ((y_predicted2 - y_test) ** 2).sum())

R2 score: 0.33
Mean absolute error  2.32
Residual sum of squares 576207.93


In [819]:
''' Calculate the Variance Inflation Factor (VIF) value. VIF is a number that 
determines whether a variable has multicollinearity or not (starts from 1, and it 
has no upper limit. If the number gets larger, it means the variable has huge 
multicollinearity on it.).'''

x=df.drop(columns="Rainfall",axis=1)
y=df['Rainfall']


vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif['variable'] = x.columns

vif

,VIF,variable
0,5.137530,Date
1,4.452700,Location
2,60.665458,MinTemp
3,614.761966,MaxTemp
4,7.307601,Evaporation
5,17.931342,Sunshine
6,6.428486,WindGustDir
7,28.795830,WindGustSpeed
8,4.687195,WindDir9am
9,6.257203,WindDir3pm


In [820]:
''' Remove multicollinearities by creating new features. Find the features that have 
paired values and create the new feature which is the difference value between 
those pairs.'''


df["Pressure3_9"]=abs(df["Pressure3pm"]-df["Pressure9am"])
df=df.drop(columns=["Pressure9am","Pressure3pm"],axis=1)

df["Humidity9_3"]=abs(df["Humidity3pm"]-df["Humidity9am"])
df=df.drop(columns=["Humidity9am","Humidity3pm"],axis=1)

df["Temp9_3"]=abs(df["Temp3pm"]-df["Temp9am"])
df=df.drop(columns=["Temp9am","Temp3pm"],axis=1)

df["Cloud9_3"]=abs(df["Cloud3pm"]-df["Cloud9am"])
df=df.drop(columns=["Cloud9am","Cloud3pm"],axis=1)

df["WindSpeed9_3"]=abs(df["WindSpeed9am"]-df["WindSpeed3pm"])
df=df.drop(columns=["WindSpeed9am","WindSpeed3pm"],axis=1)

df["Evap_Suns"]=abs(df["Evaporation"]-df["Sunshine"])
df=df.drop(columns=["Evaporation","Sunshine"],axis=1)

df["Max_MinTemp"]=abs(df["MaxTemp"]-df["MinTemp"])
df=df.drop(columns=["MaxTemp","MinTemp"],axis=1)


df

,Date,Location,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,RainToday,RainTomorrow,Pressure3_9,Humidity9_3,Temp9_3,Cloud9_3,WindSpeed9_3,Evap_Suns,Max_MinTemp
6049,1,4,0.0,11,48.0,1,12,0,0,1.9,7.0,6.8,3.0,14.0,0.3,17.3
6050,1,4,0.0,8,37.0,10,10,0,0,0.8,22.0,6.7,0.0,0.0,1.8,10.5
6052,1,4,0.0,5,46.0,5,6,0,0,3.1,20.0,6.2,5.0,15.0,0.2,18.2
6053,1,4,0.0,14,31.0,14,15,0,0,3.6,15.0,6.5,4.0,0.0,0.8,16.5
6054,1,4,0.0,14,35.0,7,14,0,0,3.3,4.0,4.0,5.0,4.0,2.8,16.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142298,6,6,0.0,1,35.0,9,4,0,0,3.4,31.0,7.8,1.0,11.0,5.0,14.1
142299,6,6,0.0,0,37.0,9,9,0,0,3.4,28.0,7.2,7.0,2.0,1.0,11.4
142300,6,6,0.0,0,33.0,0,13,0,0,3.5,23.0,7.3,0.0,6.0,5.4,12.1
142301,6,6,0.0,2,26.0,9,6,0,0,4.2,4.0,4.4,0.0,8.0,4.4,12.3


In [821]:
x=df.drop(columns="Rainfall",axis=1)
y=df['Rainfall']


vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif['variable'] = x.columns

vif

,VIF,variable
0,4.495736,Date
1,3.914641,Location
2,6.334695,WindGustDir
3,10.274407,WindGustSpeed
4,4.051396,WindDir9am
5,6.117672,WindDir3pm
6,1.616378,RainToday
7,1.595058,RainTomorrow
8,5.749237,Pressure3_9
9,6.283768,Humidity9_3


In [822]:
'''Remove features that have a VIF value above 5'''
df=df.drop(columns=["Pressure3_9","Humidity9_3","Temp9_3","Max_MinTemp"],axis=1)
df

,Date,Location,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,RainToday,RainTomorrow,Cloud9_3,WindSpeed9_3,Evap_Suns
6049,1,4,0.0,11,48.0,1,12,0,0,3.0,14.0,0.3
6050,1,4,0.0,8,37.0,10,10,0,0,0.0,0.0,1.8
6052,1,4,0.0,5,46.0,5,6,0,0,5.0,15.0,0.2
6053,1,4,0.0,14,31.0,14,15,0,0,4.0,0.0,0.8
6054,1,4,0.0,14,35.0,7,14,0,0,5.0,4.0,2.8
...,...,...,...,...,...,...,...,...,...,...,...,...
142298,6,6,0.0,1,35.0,9,4,0,0,1.0,11.0,5.0
142299,6,6,0.0,0,37.0,9,9,0,0,7.0,2.0,1.0
142300,6,6,0.0,0,33.0,0,13,0,0,0.0,6.0,5.4
142301,6,6,0.0,2,26.0,9,6,0,0,0.0,8.0,4.4


In [823]:
''' Build a regression model to perform the Rain prediction. Also, tabulate accuracy 
of the prediction models, before and VIF computation.'''


x1=df.drop(columns="Rainfall",axis=1).values
y1=df['Rainfall'].values


x_train, x_test, y_train, y_test = train_test_split(x1, y1, test_size=0.3, random_state=1)


lm = LinearRegression()
lm.fit(x_train,y_train)

y_predicted=lm.predict(x_test)

print("R2 score Before VIF: %.2f" % r2_score(y_test,y_predicted2))
print("Mean absolute error Before VIF  %.2f" %mean_absolute_error(y_test,y_predicted2))
print("Residual sum of squares Before VIF %.2f"
  % ((y_predicted2 - y_test) ** 2).sum())

print("")
print("")

print("R2 score After VIF: %.2f" % r2_score(y_test,y_predicted))
print("Mean absolute error After VIF  %.2f" %mean_absolute_error(y_test,y_predicted))
print("Residual sum of squares after VIF %.2f"
  % ((y_predicted - y_test) ** 2).sum())


R2 score Before VIF: 0.33
Mean absolute error Before VIF  2.32
Residual sum of squares Before VIF 576207.93


R2 score After VIF: 0.32
Mean absolute error After VIF  2.15
Residual sum of squares after VIF 592970.34
